<a href="https://colab.research.google.com/github/manjeet704/Datascience/blob/prac_code/Tranformer_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformes

In [ ]:
from transformers import pipeline

In [ ]:
Classifier=pipeline('sentiment-analysis')

In [ ]:
Classifier('We are very happy to show you the transformer library')

[{'label': 'POSITIVE', 'score': 0.9998239874839783}]

In [ ]:
results=Classifier(["The food is not good","The food is good"])
for result in results:
  print(f"label: {result['label']}, with score: {round(result['score'],4)}")

label: NEGATIVE, with score: 0.9997
label: POSITIVE, with score: 0.9999


In [ ]:
classifier = pipeline('sentiment-analysis', model = 'nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
classifier('The food is good')

[{'label': '4 stars', 'score': 0.4961555004119873}]

In [ ]:
classifier('Das Essen ist gut') # German language

[{'label': '4 stars', 'score': 0.5022420287132263}]

In [ ]:
classifier('La nourriture est bonne') # French language

[{'label': '4 stars', 'score': 0.4808807075023651}]

In [ ]:
#same thing to do in local
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [ ]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt = True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model = model_name, tokenizer = tokenizer)

In [ ]:
classifier('the movie was great')

In [ ]:
classifier('Food is awesome')

In [ ]:
classifier('In Cricket team Batsman are good , but the Bowler doing not very well')

[{'label': '3 stars', 'score': 0.6007073521614075}]

# Questions_ans


In [ ]:
!pip install simpletransformers

In [2]:
import json
with open(r"/content/train.json", "r") as read_file:
    train = json.load(read_file)

In [ ]:
train


In [4]:
with open(r"/content/test.json", "r") as read_file:
    test = json.load(read_file)

In [ ]:
test

In [6]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [7]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [8]:
# Configure the model
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5

In [11]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size": 3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [ ]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args , use_cuda=False
)

In [15]:
### Remove output folder
!rm -rf outputs

In [17]:
# Train the model
model.train_model(train, eval_data=test)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_init.py", line 1172, in init
    wi.setup(kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_init.py", line 306, in setup
    wandb_login._login(
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_login.py", line 317, in _login
    wlogin.prompt_api_key()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_login.py", line 240, in prompt_api_key
    key, status = self._prompt_api_key()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_login.py", line 220, in _prompt_api_key
    key = apikey.prompt_api_key(
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/lib/apikey.py", line 151, in prompt_api_key
    key = input_callback(api_ask).strip()
  File "/usr/local/lib/python3.10/dist-packages/click/termui.py", line 164, in prompt
    value = prompt_func(prompt)
  File "/usr/local/lib/python3.10/dist-packages/click/termui.py", l

Error: An unexpected error occurred

In [ ]:
# Evaluate the model
result, texts = model.eval_model(test)

In [ ]:
# Make predictions with the model
to_predict = [
    {
        "context": "Vin is a Mistborn of great power and skill.",
        "qas": [
            {
                "question": "What is Vin's speciality?",
                "id": "0",
            }
        ],
    }
]

In [ ]:
answers, probabilities = model.predict(to_predict)

print(answers)